In [3]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from typing import Optional, Callable
import wandb
from sklearn.metrics import accuracy_score

from train.train import train, evaluate
from models.LeNet import LeNet
from data.get_data import get_dataloader

In [ ]:
run = wandb.init(project="test")

In [4]:
tr, val, te = get_dataloader(dataset='MNIST', batch_size=128, num_workers=4)
len(tr), len(val), len(te)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:12<00:00, 762876.70it/s] 


Extracting data\datasets\MNIST\raw\train-images-idx3-ubyte.gz to data\datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 201930.18it/s]


Extracting data\datasets\MNIST\raw\train-labels-idx1-ubyte.gz to data\datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1871782.36it/s]


Extracting data\datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to data\datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1358907.82it/s]

Extracting data\datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\datasets\MNIST\raw



(375, 94, 79)

In [5]:
for i in tr:
    img, label = i
    break

img.shape, label.shape


(torch.Size([128, 1, 28, 28]), torch.Size([128]))

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LeNet(num_channels=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [8]:
train(  model,
        criterion,
        optimizer, 
        te,
        val_loader=val,
        device=device,
        max_epochs=1,
        eval_every=50,
        eval_metric=accuracy_score,
        # wandb_run=run
    )

Train Epoch 0/1  Batch 0/79:	
Loss: 2.415232	Score: 0.0859375
Validation:	


  0%|          | 0/79 [00:00<?, ?it/s]

In [9]:
run.finish()

train_loss,█▄▄▃▃▂▃▂▂▂▁▁▂▂▁▁▂▂▁▂▂▁▂▂▂▁▁▂▁▂▁▂▁▂▂▁▁▁▂▁
train_loss,2.50846
train_score,nan


In [45]:
for X_batch, y_batch in te:
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    output = model(X_batch)
    # y_batch = y_batch.unsqueeze(1).to(torch.float32)

    print(y_batch.shape, output.shape)
    criterion(output, y_batch)
    print(accuracy_score(y_batch.cpu().numpy(), output.argmax(1).cpu().numpy()))
    break

torch.Size([128]) torch.Size([128, 10])
0.9765625
